# TNT Single-Mode: Startup Shear
> **Handbook:** This notebook demonstrates the TNT Single-Mode model. For complete mathematical derivations and theoretical background, see [TNT Single-Mode Documentation](../../docs/source/models/tnt/index.rst).


**Estimated Time:** 3-5 minutes

## Protocol: Startup Shear Flow

**Startup shear** is a transient rheological test where a constant shear rate is suddenly applied to a material initially at rest. The time-dependent stress response reveals information about network microstructure and relaxation dynamics.

### Physical Context for Transient Networks

In associative polymer networks with reversible crosslinks:

1. **t = 0⁺**: Sudden shear rate applied → network strands begin stretching elastically
2. **0 < t < τ_b**: Stress increases as conformation tensor $\mathbf{S}$ evolves under affine deformation
3. **t ≈ τ_b**: **Stress overshoot** occurs when bond breakage rate catches up with creation rate
4. **t → ∞**: Steady state reached when $d\mathbf{S}/dt = 0$

The **stress overshoot peak** provides a direct signature of the crosslink lifetime $\tau_b$. For TNT Single-Mode with constant breakage, the peak time scales as:

$$
t_{peak} \approx \tau_b
$$

This makes startup shear an ideal protocol for extracting $\tau_b$ from a single transient experiment.

### Why TNT Single-Mode for Startup?

The Tanaka-Edwards model (constant breakage) predicts:
- **Single-exponential** stress growth to steady state
- Overshoot ratio **independent of shear rate** (linear viscoelasticity signature)
- Identical to **Upper Convected Maxwell (UCM)** model behavior

For materials showing **rate-dependent overshoot** or **multiple relaxation timescales**, see:
- **Cates model (NB08)**: Living polymers with geometric mean relaxation $\tau_d = \sqrt{\tau_{rep} \tau_{break}}$
- **Sticky Rouse (NB26)**: Multi-mode sticker dynamics with hierarchical relaxation

> **TNT Startup Equations**  
> For the conformation tensor ODE integration and stress computation, see:  
> [../../docs/source/models/tnt/tnt_protocols.rst](../../docs/source/models/tnt/tnt_protocols.rst) — Section on "Startup Shear"

---

## Learning Objectives

1. Understand stress overshoot in transient network materials
2. Fit TNT single-mode model to startup shear data
3. Analyze conformation tensor evolution during startup
4. Predict stress overshoot at different shear rates
5. Extract network relaxation time from overshoot dynamics
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- Understanding of transient network theory (Notebook 01)
- Familiarity with time-dependent rheology
- Knowledge of stress overshoot phenomena

## Runtime Estimate

- NLSQ fitting: ~3-8 seconds
- Bayesian inference (demo): ~45-90 seconds
- Total: ~3-5 minutes

## 1. Setup

In [ ]:
import os
import sys
import time

# Google Colab support
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTSingleMode

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_pnas_startup,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_single_mode_param_names,
    plot_conformation_tensor_evolution,
    get_bayesian_config,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## 2. Theory

## Stress Overshoot in Transient Networks

### Physical Mechanism

When a sudden shear rate is applied to a transient network:
1. **Initial phase**: Network strands stretch elastically → stress increases rapidly
2. **Overshoot**: Stress reaches maximum when strand creation/breakage balance is disrupted
3. **Relaxation**: Network reorganizes, stress decreases to steady-state value

### Governing Equations

The conformation tensor **S** evolves according to:

$$
\frac{dS}{dt} = \mathbf{L} \cdot \mathbf{S} + \mathbf{S} \cdot \mathbf{L}^T + g_0 \mathbf{I} - \beta \mathbf{S}
$$

Stress is computed from:

$$
\boldsymbol{\sigma} = G (\mathbf{S} - \mathbf{I}) + \eta_s \mathbf{D}
$$

where $\mathbf{D}$ is the rate of deformation tensor.

### Steady-State Stress

At steady state (t → ∞):

$$
\sigma_{ss} = G \tau_b \dot{\gamma} + \eta_s \dot{\gamma}
$$

### Overshoot Characteristics

- **Peak time**: Occurs at $t_{peak} \approx \tau_b$ (one relaxation time)
- **Peak stress**: $\sigma_{peak} > \sigma_{ss}$
- **Overshoot ratio**: $\sigma_{peak}/\sigma_{ss}$ depends on Wi = $\dot{\gamma} \tau_b$

### Parameters

| Parameter | Symbol | Physical Meaning | Typical Range |
|-----------|--------|------------------|---------------|
| Elastic modulus | $G$ | Network stiffness | 1-1000 Pa |
| Breakage time | $\tau_b$ | Crosslink lifetime | 0.01-100 s |
| Solvent viscosity | $\eta_s$ | Background viscosity | 0.001-1 Pa·s |

## 3. Load Data

In [ ]:
# Load startup shear data
gamma_dot = 1.0  # s^-1
t_data, stress = load_pnas_startup(gamma_dot=gamma_dot)

print(f"Data shape: {len(t_data)} points")
print(f"Time range: {t_data.min():.3f} - {t_data.max():.3f} s")
print(f"Stress range: {stress.min():.2f} - {stress.max():.2f} Pa")
print(f"Applied shear rate: {gamma_dot} s⁻¹")

In [ ]:
# Plot raw data
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(t_data, stress, 'o', label='Data', markersize=6, alpha=0.7)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel(r'Shear stress $\sigma$ (Pa)', fontsize=12)
ax.set_title(f'Startup Shear at γ̇ = {gamma_dot} s⁻¹', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close("all")

# Identify overshoot
peak_idx = np.argmax(stress)
peak_time = t_data[peak_idx]
peak_stress = stress[peak_idx]
steady_stress = np.mean(stress[-10:])
overshoot_ratio = peak_stress / steady_stress

print(f"\nOvershoot Analysis:")
print(f"  Peak stress: {peak_stress:.2f} Pa at t = {peak_time:.3f} s")
print(f"  Steady-state stress: {steady_stress:.2f} Pa")
print(f"  Overshoot ratio: {overshoot_ratio:.2f}")

## 4. NLSQ Fitting

In [ ]:
# Create model and fit
model = TNTSingleMode(breakage="constant")

start_time = time.time()
result = model.fit(t_data, stress, test_mode="startup", gamma_dot=gamma_dot, method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ fitting completed in {fit_time:.2f} seconds")
print(f"\nFitted Parameters:")
param_names = get_tnt_single_mode_param_names(breakage="constant")
for name in param_names:
    param = model.parameters.get(name)
    print(f"  {name} = {param.value:.4e}")

In [ ]:
# Compute fit quality metrics
metrics = compute_fit_quality(stress, model.predict(t_data, test_mode="startup", gamma_dot=gamma_dot))
print(f"\nFit Quality:")
print(f"  R² = {metrics['R2']:.6f}")
print(f"  RMSE = {metrics['RMSE']:.4e} Pa")
print(f"  NRMSE = {metrics['NRMSE']*100:.2f}%")

In [ ]:
# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    t_data, stress, model, test_mode="startup",
    param_names=param_names, log_scale=False,
    xlabel='Time (s)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R\u00b2 = {metrics["R2"]:.4f})',
    gamma_dot=gamma_dot
)
plt.close("all")

### 4.2 Physical Analysis

In [ ]:
time_fine = np.linspace(t_data.min(), t_data.max(), 200)
stress_pred = model.simulate_startup(time_fine, gamma_dot=gamma_dot)
# Extract fitted parameters
G = model.parameters.get('G').value
tau_b = model.parameters.get('tau_b').value
eta_s = model.parameters.get('eta_s').value

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Multi-rate startup predictions
ax = axes[0, 0]
gamma_dot_list = [0.1, 1.0, 10.0]
colors = ['C0', 'C1', 'C2']
for gd, color in zip(gamma_dot_list, colors):
    stress_i = model.simulate_startup(time_fine, gamma_dot=gd)
    ax.plot(time_fine, stress_i, '-', linewidth=2, color=color, label=f'γ̇ = {gd} s⁻¹')
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel(r'Shear stress $\sigma$ (Pa)', fontsize=11)
ax.set_title('Multi-Rate Startup', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 2. Normalized stress evolution
ax = axes[0, 1]
for gd, color in zip(gamma_dot_list, colors):
    stress_i = model.simulate_startup(time_fine, gamma_dot=gd)
    sigma_ss_i = G * tau_b * gd + eta_s * gd
    ax.plot(time_fine / tau_b, stress_i / sigma_ss_i, '-', linewidth=2, color=color, label=f'γ̇ = {gd} s⁻¹')
ax.axhline(y=1, color='k', linestyle='--', alpha=0.5)
ax.set_xlabel(r'Dimensionless time $t/\tau_b$', fontsize=11)
ax.set_ylabel(r'$\sigma / \sigma_{ss}$', fontsize=11)
ax.set_title('Normalized Stress Evolution', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 3. Overshoot analysis
ax = axes[1, 0]
gamma_dot_sweep = np.logspace(-1, 2, 20)
overshoot_ratios = []
peak_times = []

for gd in gamma_dot_sweep:
    t_test = np.linspace(0, 10*tau_b, 500)
    stress_test = model.simulate_startup(t_test, gamma_dot=gd)
    sigma_ss_test = G * tau_b * gd + eta_s * gd
    peak_idx_test = np.argmax(stress_test)
    overshoot_ratios.append(stress_test[peak_idx_test] / sigma_ss_test)
    peak_times.append(t_test[peak_idx_test])

ax.semilogx(gamma_dot_sweep * tau_b, overshoot_ratios, 'o-', linewidth=2, markersize=6)
ax.axhline(y=1, color='k', linestyle='--', alpha=0.5)
ax.set_xlabel(r'Weissenberg number $Wi = \dot{\gamma}\tau_b$', fontsize=11)
ax.set_ylabel(r'Overshoot ratio $\sigma_{peak}/\sigma_{ss}$', fontsize=11)
ax.set_title('Overshoot vs. Weissenberg Number', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# 4. Peak time scaling
ax = axes[1, 1]
ax.loglog(gamma_dot_sweep * tau_b, np.array(peak_times) / tau_b, 'o-', linewidth=2, markersize=6)
ax.axhline(y=1, color='k', linestyle='--', alpha=0.5, label=r'$t_{peak} = \tau_b$')
ax.set_xlabel(r'Weissenberg number $Wi = \dot{\gamma}\tau_b$', fontsize=11)
ax.set_ylabel(r'Dimensionless peak time $t_{peak}/\tau_b$', fontsize=11)
ax.set_title('Peak Time Scaling', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")

print(f"\nPhysical Interpretation:")
print(f"  Network modulus: G = {G:.2f} Pa")
print(f"  Breakage time: τ_b = {tau_b:.4f} s")
print(f"  Peak occurs at t ≈ τ_b = {tau_b:.4f} s (one relaxation time)")
print(f"  Steady-state stress: σ_ss = G·τ_b·γ̇ + η_s·γ̇ = {G*tau_b*gamma_dot + eta_s*gamma_dot:.2f} Pa")

## 5. Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        t_data, stress,
        test_mode='startup',
        gamma_dot=gamma_dot,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(bayesian_result, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayesian_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        t_data,
        stress,
        model, bayesian_result, test_mode="startup",
        param_names=param_names, log_scale=False,
        xlabel=r'Time (s)',
        ylabel=r'Shear stress $\\sigma$ (Pa)', gamma_dot=gamma_dot
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

In [ ]:
# Parameter comparison table
print_parameter_comparison(model, bayesian_result.posterior_samples, param_names)

## 6. Physical Interpretation

### Stress Overshoot Mechanism

The stress overshoot arises from the **transient imbalance** between network creation and breakage:

1. **t = 0**: Network is at equilibrium, sudden shear applied
2. **0 < t < τ_b**: Network strands stretch elastically, stress increases linearly
3. **t ≈ τ_b**: Overshoot peak — maximum strand extension before significant breakage
4. **t > τ_b**: Breakage rate increases, network reorganizes, stress relaxes to steady state

### Peak Time Scaling

For TNT models, the peak time is approximately:

$$
t_{peak} \approx \tau_b
$$

This provides a direct measure of the **crosslink lifetime** from a single startup experiment.

### Overshoot Ratio

The overshoot ratio $\sigma_{peak}/\sigma_{ss}$ depends on the Weissenberg number:
- Low Wi: Small overshoot (network equilibrates quickly)
- High Wi: Large overshoot (network stretches significantly before breakage)

For constant breakage TNT, the overshoot ratio is **independent of Wi** — a signature of linear viscoelasticity.

### Comparison with Maxwell Model

The TNT constant breakage model reduces to the Upper Convected Maxwell (UCM) model:
- Predicts exponential stress growth
- Overshoot ratio is constant (~2.7 for simple shear)
- Peak time = τ_b

### Material Applications

Startup shear is ideal for characterizing:
- **Associative polymers**: Telechelic polymers, ionomers
- **Supramolecular networks**: Hydrogen-bonded systems
- **Colloidal gels**: Depletion gels with reversible attractions

## 7. Save Results

In [ ]:
# Save results
save_tnt_results(model, bayesian_result, "single_mode", "startup", param_names)
print("\nResults saved successfully!")

## Key Takeaways

1. **Stress overshoot**: Signature of transient network stretching and relaxation
2. **Peak time**: t_peak ≈ τ_b provides direct measure of crosslink lifetime
3. **Overshoot ratio**: σ_peak/σ_ss depends on Weissenberg number
4. **Steady state**: σ_ss = G·τ_b·γ̇ + η_s·γ̇ (same as flow curve)
5. **Bayesian inference**: Quantifies uncertainty in transient dynamics
6. **Multi-rate analysis**: Reveals scaling behavior with deformation rate

## Next Steps

- **Notebook 03**: Stress relaxation for exponential decay
- **Notebook 04**: Creep response for compliance analysis
- **Notebook 05**: SAOS for frequency-dependent moduli
- **Notebook 06**: LAOS for nonlinear harmonics

## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
